# What Happens When the Wrong Operations are Performed?

In our tests we trust the prepartory system (Alice). We assume Alice properly performs the correct operations and prepares the desired states. However, we do not claim to know if Bob can properly perform the required tests. What happens if Bob does not perform the correct operations? This document runs through examples of performing the wrong measurements and how our tests detect this.

# Dimensionality Testing

# Incompatible Measurement Testing

This module determines if a quantum device can perform incompatible measurements. Recall that incompatible measurments are those which do not commute, measuring in the Z and X bases, for example. Below the compatible measurement from the theory notebook is performed. Bob's first (y=0) projective measurement performs the following rotation (the global phase on the second basis state has been changed):
\begin{pmatrix}
    cos(\theta) & sin(\theta)\\
    -sin(\theta) & cos(\theta)\\
\end{pmatrix}
Thus we can perform Bob's measurement via a Y rotation by - $2\theta$ then a measurment in the computational basis. Now let's construct the matrix for the compatible measurement. This is given by:
\begin{pmatrix}
    sin(\theta + \pi /4) & -cos(\theta + \pi /4)\\
    cos(\theta + \pi /4) & sin(\theta + \pi /4)\\
\end{pmatrix}
Using a couple trig properties, this becomes
\begin{pmatrix}
    cos(\pi /4 -\theta) & -sin(\pi /4 -\theta)\\
    sin(\pi / 4 -\theta) & cos(\pi /4 -\theta)\\
\end{pmatrix}
The above matrix is a rotation of $2(\pi /4 -\theta)$ around the Y axis. In the incompatible measurement test, we use $\theta = \pi / 8$. Thus, this measurement is achieved via a rotation of $\pi / 4$.<br><br>
The code snippet below implements this compatible measurement. This set of projective measurements should not violate the classical bounds.

In [1]:
from qiskit import QuantumCircuit, IBMQ
import numpy as np
import context
from device_independent_test import incompatible_measurement
from device_independent_test import quantum_communicator
provider = IBMQ.load_account()

dispatch = quantum_communicator.LocalDispatcher([provider.get_backend('ibmq_qasm_simulator')])

# create bb84 states
pre_ops = [incompatible_measurement.bb84_states()]

# measure in the wrong basis
qc0 = QuantumCircuit(4)
qc0.u3(-np.pi/4,0,0,range(0,4))
qc0.measure_all()

qc1 = QuantumCircuit(4)
qc1.u3(np.pi/4,0,0,range(0,4)) # compatible basis
qc1.measure_all()

post_ops = [[QuantumCircuit(4)],[qc0,qc1]]
counts = dispatch.batch_run_and_transmit(pre_ops,post_ops,1000)
violation = incompatible_measurement.bell_violation(counts[0],counts[1],1000,1000)

print("Incompatible measurement test score: ", violation)
print("Classical bound is 6. Quantum wystems score up to about 6.8.")

┌───────────────┐                                   ░ ┌─┐         
   q_0: ┤ U3(-pi/4,0,0) ├───────────────────────────────────░─┤M├─────────
        └─────┬───┬─────┘┌───────────────┐                  ░ └╥┘┌─┐      
   q_1: ──────┤ X ├──────┤ U3(-pi/4,0,0) ├──────────────────░──╫─┤M├──────
              ├───┤      ├───────────────┤                  ░  ║ └╥┘┌─┐   
   q_2: ──────┤ H ├──────┤ U3(-pi/4,0,0) ├──────────────────░──╫──╫─┤M├───
              ├───┤      └─────┬───┬─────┘┌───────────────┐ ░  ║  ║ └╥┘┌─┐
   q_3: ──────┤ X ├────────────┤ H ├──────┤ U3(-pi/4,0,0) ├─░──╫──╫──╫─┤M├
              └───┘            └───┘      └───────────────┘ ░  ║  ║  ║ └╥┘
meas_0: ═══════════════════════════════════════════════════════╩══╬══╬══╬═
                                                                  ║  ║  ║ 
meas_1: ══════════════════════════════════════════════════════════╩══╬══╬═
                                                                     ║  ║ 
meas_2: ═════════════════════════════════════════════════════════════╩══╬═
                                                                        ║ 
meas_3: ════════════════════════════════════════════════════════════════╩═

┌──────────────┐                                 ░ ┌─┐         
   q_0: ┤ U3(pi/4,0,0) ├─────────────────────────────────░─┤M├─────────
        └────┬───┬─────┘┌──────────────┐                 ░ └╥┘┌─┐      
   q_1: ─────┤ X ├──────┤ U3(pi/4,0,0) ├─────────────────░──╫─┤M├──────
             ├───┤      ├──────────────┤                 ░  ║ └╥┘┌─┐   
   q_2: ─────┤ H ├──────┤ U3(pi/4,0,0) ├─────────────────░──╫──╫─┤M├───
             ├───┤      └────┬───┬─────┘┌──────────────┐ ░  ║  ║ └╥┘┌─┐
   q_3: ─────┤ X ├───────────┤ H ├──────┤ U3(pi/4,0,0) ├─░──╫──╫──╫─┤M├
             └───┘           └───┘      └──────────────┘ ░  ║  ║  ║ └╥┘
meas_0: ════════════════════════════════════════════════════╩══╬══╬══╬═
                                                               ║  ║  ║ 
meas_1: ═══════════════════════════════════════════════════════╩══╬══╬═
                                                                  ║  ║ 
meas_2: ══════════════════════════════════════════════════════════╩══╬═
                                                                     ║ 
meas_3: ═════════════════════════════════════════════════════════════╩═

Incompatible Measurement Test Score:  4.054
Classical Bound is 6. Quantum Systems score up to about 6.8.


# Entanglement Testing

The CHSH test in this library truly relies both upon entanglement and incompatible measurments. For the purposes of this document, we will assume that the ability to perform incompatible measurments has already been verified. The code below demonstrates what happens if an entangled pair is not generated.

The first example generates random states and performs the CHSH measurments upon them (Alice projectively measuring in the Z and X bases, Bob in the W and V). A quantum system with entanglement should be able to achieve a test score of magnitude greater than 2 (the classical bound).

In [3]:
import context
from device_independent_test import entanglement
from qiskit.quantum_info import random_unitary

dispatch = quantum_communicator.LocalDispatcher([provider.get_backend('ibmq_qasm_simulator')])
shots = 1000

# Generate two random states via random unitary operations
random_circ = random_unitary(2).tensor(random_unitary(2))
qc = QuantumCircuit(2)
qc.unitary(random_circ,[0,1],label='random unitaries')
qc.draw()

post_ops = [[QuantumCircuit(2)],[entanglement.measure_in_ZW(),entanglement.measure_in_ZV(),entanglement.measure_in_XW(),entanglement.measure_in_XV()]]
counts = dispatch.batch_run_and_transmit([qc],post_ops,shots)

expected_ZW = entanglement.compute_expectation_for_CHSH(counts[0], shots)
expected_ZV = entanglement.compute_expectation_for_CHSH(counts[1], shots)
expected_XW = entanglement.compute_expectation_for_CHSH(counts[2], shots)
expected_XV = entanglement.compute_expectation_for_CHSH(counts[3], shots)

test_val = (expected_ZW + expected_ZV + expected_XW - expected_XV)
print(test_val)

AttributeError: module 'device_independent_test.entanglement' has no attribute 'measure_in_ZW'

In [8]:
import context
from device_independent_test import entanglement
from random import random

dispatch = quantum_communicator.LocalDispatcher([provider.get_backend('ibmq_qasm_simulator')])
shots = 200
runs = 5
p_failure = 0.2

# Alice's measurements (Z or X basis)
alice_z = entanglement.alice_Z()
alice_z.measure(0,0)
alice_x = entanglement.alice_X()
alice_x.measure(0,0)

# Bob's measurements (W or V basis)
bob_w = entanglement.bob_W()
bob_w.measure(1,1)
bob_v = entanglement.bob_V()
bob_v.measure(1,1)

post_ops = [[alice_z,alice_x],[bob_w,bob_v]]
counts = [{"00":0,"11":0,"10":0,"01":0},{"00":0,"11":0,"10":0,"01":0},{"00":0,"11":0,"10":0,"01":0},{"00":0,"11":0,"10":0,"01":0}]

for i in range(0,runs):
    qc = QuantumCircuit(2)
    qc.h(0)
    if random() > p_failure:
        qc.cx(0,1)
    temp = dispatch.batch_run_and_transmit([qc],post_ops,shots)
    for j in range (0,4):
        for key in counts[j]:
            counts[j][key] = counts[j][key]+temp[j].get(key,0.0)

expected_ZW = entanglement.compute_expectation_for_CHSH(counts[0], shots*runs)
expected_ZV = entanglement.compute_expectation_for_CHSH(counts[1], shots*runs)
expected_XW = entanglement.compute_expectation_for_CHSH(counts[2], shots*runs)
expected_XV = entanglement.compute_expectation_for_CHSH(counts[3], shots*runs)

test_val = (expected_ZW + expected_ZV + expected_XW - expected_XV)
print(test_val)


┌───┐┌─┐                  
q_0: ┤ H ├┤M├──────────────────
     ├───┤└╥┘┌───┐┌───┐┌───┐┌─┐
q_1: ┤ S ├─╫─┤ H ├┤ T ├┤ H ├┤M├
     └───┘ ║ └───┘└───┘└───┘└╥┘
c_0: ══════╩═════════════════╬═
                             ║ 
c_1: ════════════════════════╩═

┌───┐┌─┐                    
q_0: ┤ H ├┤M├────────────────────
     ├───┤└╥┘┌───┐┌─────┐┌───┐┌─┐
q_1: ┤ S ├─╫─┤ H ├┤ TDG ├┤ H ├┤M├
     └───┘ ║ └───┘└─────┘└───┘└╥┘
c_0: ══════╩═══════════════════╬═
                               ║ 
c_1: ══════════════════════════╩═

┌───┐┌───┐┌─┐             
q_0: ┤ H ├┤ H ├┤M├─────────────
     ├───┤├───┤└╥┘┌───┐┌───┐┌─┐
q_1: ┤ S ├┤ H ├─╫─┤ T ├┤ H ├┤M├
     └───┘└───┘ ║ └───┘└───┘└╥┘
c_0: ═══════════╩════════════╬═
                             ║ 
c_1: ════════════════════════╩═

┌───┐┌───┐┌─┐               
q_0: ┤ H ├┤ H ├┤M├───────────────
     ├───┤├───┤└╥┘┌─────┐┌───┐┌─┐
q_1: ┤ S ├┤ H ├─╫─┤ TDG ├┤ H ├┤M├
     └───┘└───┘ ║ └─────┘└───┘└╥┘
c_0: ═══════════╩══════════════╬═
                               ║ 
c_1: ══════════════════════════╩═

┌───┐     ┌─┐                       
q_0: ┤ H ├──■──┤M├───────────────────────
     └───┘┌─┴─┐└╥┘┌───┐┌───┐┌───┐┌───┐┌─┐
q_1: ─────┤ X ├─╫─┤ S ├┤ H ├┤ T ├┤ H ├┤M├
          └───┘ ║ └───┘└───┘└───┘└───┘└╥┘
c_0: ═══════════╩══════════════════════╬═
                                       ║ 
c_1: ══════════════════════════════════╩═

...

┌───┐     ┌─┐                       
q_0: ┤ H ├──■──┤M├───────────────────────
     └───┘┌─┴─┐└╥┘┌───┐┌───┐┌───┐┌───┐┌─┐
q_1: ─────┤ X ├─╫─┤ S ├┤ H ├┤ T ├┤ H ├┤M├
          └───┘ ║ └───┘└───┘└───┘└───┘└╥┘
c_0: ═══════════╩══════════════════════╬═
                                       ║ 
c_1: ══════════════════════════════════╩═

┌───┐     ┌─┐                         
q_0: ┤ H ├──■──┤M├─────────────────────────
     └───┘┌─┴─┐└╥┘┌───┐┌───┐┌─────┐┌───┐┌─┐
q_1: ─────┤ X ├─╫─┤ S ├┤ H ├┤ TDG ├┤ H ├┤M├
          └───┘ ║ └───┘└───┘└─────┘└───┘└╥┘
c_0: ═══════════╩════════════════════════╬═
                                         ║ 
c_1: ════════════════════════════════════╩═

┌───┐     ┌───┐┌─┐                  
q_0: ┤ H ├──■──┤ H ├┤M├──────────────────
     └───┘┌─┴─┐├───┤└╥┘┌───┐┌───┐┌───┐┌─┐
q_1: ─────┤ X ├┤ S ├─╫─┤ H ├┤ T ├┤ H ├┤M├
          └───┘└───┘ ║ └───┘└───┘└───┘└╥┘
c_0: ════════════════╩═════════════════╬═
                                       ║ 
c_1: ══════════════════════════════════╩═

┌───┐     ┌───┐┌─┐                    
q_0: ┤ H ├──■──┤ H ├┤M├────────────────────
     └───┘┌─┴─┐├───┤└╥┘┌───┐┌─────┐┌───┐┌─┐
q_1: ─────┤ X ├┤ S ├─╫─┤ H ├┤ TDG ├┤ H ├┤M├
          └───┘└───┘ ║ └───┘└─────┘└───┘└╥┘
c_0: ════════════════╩═══════════════════╬═
                                         ║ 
c_1: ════════════════════════════════════╩═

2.178
